In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install grad-cam
!pip install pytorch-gradcam

In [ ]:
import torch
from pytorch_lightning import Trainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.listdir("drive/MyDrive/Online_courses/ERA/S13/PASCAL_VOC")

['100examples.csv',
 '1examples.csv',
 '2examples.csv',
 '8examples.csv',
 'train.csv',
 'test.csv',
 'images',
 'labels',
 'label_sample',
 'image_sample']

In [ ]:
#from model import *
#from utils import *

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from pytorch_lightning import LightningModule, Trainer

import config
from loss import YoloLoss
from dataset import YOLODataset

"""
Information about architecture config:
Tuple is structured by (filters, kernel_size, stride)
Every conv is a same convolution.
List is structured by "B" indicating a residual block followed by the number of repeats
"S" is for scale prediction block and computing the yolo loss
"U" is for upsampling the feature map and concatenating with a previous layer
"""
layers_config = [
    (32, 3, 1),
    (64, 3, 2),
    ["B", 1],
    (128, 3, 2),
    ["B", 2],
    (256, 3, 2),
    ["B", 8],
    (512, 3, 2),
    ["B", 8],
    (1024, 3, 2),
    ["B", 4],  # To this point is Darknet-53
    (512, 1, 1),
    (1024, 3, 1),
    "S",
    (256, 1, 1),
    "U",
    (256, 1, 1),
    (512, 3, 1),
    "S",
    (128, 1, 1),
    "U",
    (128, 1, 1),
    (256, 3, 1),
    "S",
]


from pytorch_lightning.callbacks import LearningRateFinder


class get_lr(LearningRateFinder):
    def __init__(self, milestones, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.milestones = milestones

    def on_fit_start(self, *args, **kwargs):
        return

    def on_train_epoch_start(self, trainer, pl_module):
        if trainer.current_epoch in self.milestones or trainer.current_epoch == 0:
            self.lr_find(trainer, pl_module)



class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels = in_channels,
                              out_channels = out_channels,
                              bias = not bn_act,
                              **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky = nn.LeakyReLU()
        self.use_bn = bn_act

    def forward(self, x):
        if self.use_bn:
            out = self.leaky(self.bn(self.conv(x)))
        else:
            out = self.conv(x)
        return out


class ResBlock(nn.Module):
    def __init__(self, in_channels, use_residual = True, num_repeat = 1):
        super().__init__()
        self.layers = nn.ModuleList()
        for repeat in range(num_repeat):
            self.layers += [
                nn.Sequential(
                    ConvBlock(in_channels = in_channels,
                              out_channels = in_channels//2,
                              kernel_size=1,
                              ),
                    ConvBlock(in_channels = in_channels//2,
                              out_channels = in_channels,
                              kernel_size = 3,
                              padding = 1)
                    )
                ]
        self.use_residual = use_residual
        self.num_repeat = num_repeat

    def forward(self, x):
        for layer in self.layers:
            if self.use_residual:
                x = x + layer(x)
            else:
                x = layer(x)
        return x


class ScalePrediction(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.pred = nn.Sequential(
            ConvBlock(
                in_channels = in_channels,
                out_channels = in_channels*2,
                kernel_size = 3,
                padding = 1
                ),
            ConvBlock(
                in_channels = in_channels*2,
                out_channels = (num_classes + 5)*3,
                bn_act = False,
                kernel_size = 1
                )
            )
        self.num_classes = num_classes

    def forward(self, x):
        return (
            self.pred(x)
            .reshape(x.shape[0], 3, self.num_classes + 5, x.shape[2], x.shape[3])
            .permute(0, 1, 3, 4, 2)
            )


class LT_model(LightningModule):
    def __init__(self, learning_rate = 0.001, hidden_size = 16, in_channels = 3, num_classes = 80):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.layers = self._create_conv_layers()
        self.learning_rate = learning_rate
        self.scaled_anchors = (
            torch.tensor(config.ANCHORS)
            * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
            )

    def _create_conv_layers(self):
        layers = nn.ModuleList()
        in_channels = self.in_channels

        for module in layers_config:
            if isinstance(module, tuple):
                out_channels, kernel_size, stride = module
                layers.append(
                    ConvBlock(
                        in_channels,
                        out_channels,
                        kernel_size=kernel_size,
                        stride=stride,
                        padding=1 if kernel_size == 3 else 0,
                    )
                )
                in_channels = out_channels

            elif isinstance(module, list):
                num_repeats = module[1]
                layers.append(ResBlock(in_channels, num_repeat=num_repeats))

            elif isinstance(module, str):
                if module == "S":
                    layers += [
                        ResBlock(in_channels, use_residual=False, num_repeat=1),
                        ConvBlock(
                            in_channels = in_channels,
                            out_channels = in_channels // 2,
                            kernel_size=1
                            ),
                        ScalePrediction(in_channels // 2, num_classes=self.num_classes),
                    ]
                    in_channels = in_channels // 2

                elif module == "U":
                    layers.append(nn.Upsample(scale_factor=2),)
                    in_channels = in_channels * 3

        return layers


    def forward(self, x):
        outputs = []
        route_connection = []

        for layer in self.layers:
            if isinstance(layer, ScalePrediction): #3 images are sent - small, medium, large
                outputs.append(layer(x))
                continue

            x = layer(x)
            if isinstance(layer, ResBlock) and layer.num_repeat==8:
                route_connection.append(x)

            elif isinstance(layer, nn.Upsample):
                x = torch.cat([x, route_connection[-1]], dim=1)
                route_connection.pop()

        return outputs



    def criterion(self, out, y):
        loss_fn = YoloLoss()
        y0, y1, y2 = (
                y[0].to(config.DEVICE),
                y[1].to(config.DEVICE),
                y[2].to(config.DEVICE),
            )
        loss = (
            loss_fn(out[0].to(config.DEVICE), y0, self.scaled_anchors[0].to(config.DEVICE))
            + loss_fn(out[1].to(config.DEVICE), y1, self.scaled_anchors[1].to(config.DEVICE))
            + loss_fn(out[2].to(config.DEVICE), y2, self.scaled_anchors[2].to(config.DEVICE))
            )
        return loss


    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self.criterion(out, y)
        #self.scaler.scale(loss).backward()
        #self.scaler.step(self.optimizer)
        #self.scaler.update()
        #self.scheduler.step()
        self.log("train_loss", loss, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
         x, y = batch
         out = self(x)
         loss = self.criterion(out, y)
         self.log("val_loss", loss, prog_bar=True)
         return loss

    def test_step(self, batch, batch_idx):
         x, y = batch
         out = self(x)
         loss = self.criterion(out, y)
         return loss



    def configure_optimizers(self):
        print("Configuring Optimisers and scheduler")
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        max_lr = 1e-3
        train_loader = self.train_dataloader()
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer,
                                                             max_lr,
                                                             steps_per_epoch=len(train_loader),
                                                             epochs = config.NUM_EPOCHS * 2 // 5)
        self.scaler = torch.cuda.amp.GradScaler()


      ####################
      # DATA RELATED HOOKS
      ####################


    def prepare_data(self):

        IMAGE_SIZE = config.IMAGE_SIZE
        train_csv_path = config.TRAIN_CSV_PATH
        test_csv_path = config.TEST_CSV_PATH

        self.train_dataset = YOLODataset(
            train_csv_path,
            img_dir=config.IMG_DIR,
            label_dir=config.LABEL_DIR,
            anchors=config.ANCHORS,
            transform=config.train_transforms,
            S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        )
        print("Train dataset prepared")

        self.test_dataset = YOLODataset(
            test_csv_path,
            img_dir=config.IMG_DIR,
            label_dir=config.LABEL_DIR,
            anchors=config.ANCHORS,
            transform=config.test_transforms,
            S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        )
        print("Test dataset prepared")


        self.train_eval_dataset = YOLODataset(
            train_csv_path,
            transform=config.test_transforms,
            S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
            img_dir=config.IMG_DIR,
            label_dir=config.LABEL_DIR,
            anchors=config.ANCHORS,
        )
        print("Validation dataset prepared")

    def train_dataloader(self):
        print("Train data loader")
        train_loader = DataLoader(
            dataset=self.train_dataset,
            batch_size=config.BATCH_SIZE,
            num_workers=config.NUM_WORKERS,
            pin_memory=config.PIN_MEMORY,
            shuffle=True,
            drop_last=False,
        )
        return train_loader

    def val_dataloader(self):
        print("Validation data loader")
        train_eval_loader = DataLoader(
            dataset=self.train_eval_dataset,
            batch_size=config.BATCH_SIZE,
            num_workers=config.NUM_WORKERS,
            pin_memory=config.PIN_MEMORY,
            shuffle=False,
            drop_last=False,
        )
        return train_eval_loader


    def test_dataloader(self):
         print("Test data loader")
         test_loader = DataLoader(
             dataset=self.test_dataset,
             batch_size=config.BATCH_SIZE,
             num_workers=config.NUM_WORKERS,
             pin_memory=config.PIN_MEMORY,
             shuffle=False,
             drop_last=False,
         )
         return test_loader




In [ ]:
model = LT_model()
trainer = Trainer(max_epochs=3)
trainer.fit(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Train dataset prepared
8 2 torch.Size([3, 416, 416]) 3 torch.Size([3, 13, 13, 6])
Test dataset prepared


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:187: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 62.0 M
--------------------------------------
62.0 M    Trainable params
0         Non-trainable params
62.0 M    Total params
247.803   Total estimated model params size (MB)


Configuring Optimisers and scheduler
Train data loader


Sanity Checking: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
trainer.test()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:149: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(


Train dataset prepared
8 2 torch.Size([3, 416, 416]) 3 torch.Size([3, 13, 13, 6])


INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_18/checkpoints/epoch=2-step=3.ckpt


Test dataset prepared


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_18/checkpoints/epoch=2-step=3.ckpt


Test data loader


Testing: 0it [00:00, ?it/s]

[{}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')